In [1]:
from fastai import datasets
import pickle, gzip

In [2]:
import torch

[^1]: what is gzip module? And How do I know this is file which has extension" .gz?

In [3]:
MNIST_URL = "http://deeplearning.net/data/mnist/mnist.pkl"

In [4]:
def init_data():
    f = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(f, 'rb') as tmp:
        (train_x, train_y), (valid_x, valid_y), _ = pickle.load(tmp, encoding = 'latin-1')
    return map(torch.tensor,(train_x, train_y, valid_x, valid_y))
train_x, train_y, valid_x, valid_y = init_data()

In [5]:
mu, sigma = train_x.mean(), train_x.std()
def normalize(x, m, s): return (x-m)/s

In [6]:
train_x, valid_x = normalize(train_x, mu, sigma), normalize(valid_x, mu, sigma)

In [7]:
def describe(x): return x.mean(), x.std(), x.type()

In [8]:
describe(train_x), describe(valid_x)

((tensor(0.0001), tensor(1.), 'torch.FloatTensor'),
 (tensor(-0.0057), tensor(0.9924), 'torch.FloatTensor'))

1. with einsum
2. with torch module

In [37]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.fwd(*args)
        return(self.out)
    def backward(self):
        self.bwd(self.out, *self.args)

In [58]:
class Lin(Module):
    def __init__(self, w, b):
        self.w = w
        self.b = b
    def fwd(self, x): return(x @ self.w + self.b)
    def bwd(self, y, x):
        self.w.g = torch.einsum("ij, ik -> jk", x, y)
        self.b.g = y.sum(0)
        x.g = torch.einsum("ik, jk -> ij", y, self.w)

In [68]:
class ReLU(Module):
    def fwd(self, x): return x.clamp_min(0.) - 0.5
        # self.x = x # we don't have to save this since it's saved at Module().args
    def bwd(self, y, x):
        x.g = (y>0).float()

In [69]:
class Mse(Module):
    def fwd(self, x, trg): return (x.squeeze() - trg).pow(2).mean().unsqueeze(-1)
    def bwd(self, y, x, trg):
        x.g = 2* (x.squeeze() - trg) / x.shape[0]

In [70]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), ReLU(), Lin(w2, b2)]
        self.loss = Mse()
    def __call__(self, x, y):
        for l in self.layers:
            x = l(x)
            #x = l.fwd(x)
        return self.loss.fwd(x, y)
    def backward(self):
        for l in reversed(self.layers): l.backward()

In [71]:
m, nh, n = train_x.shape[1], 64, 1

In [72]:
from math import sqrt
w1, w2 = torch.randn(m, nh) * sqrt(2/m), torch.randn(nh, 1) * sqrt(2/nh)
b1, b2 = torch.zeros(nh), torch.zeros(1)

In [73]:
m = Model(w1, b1, w2, b2)

In [74]:
# %timeit -n 10 m(train_x, train_y)

In [75]:
loss = m(train_x, train_y)
m.backward()

In [76]:
loss

tensor([33.9851])

In [79]:
w1.stat, w2.stat, b1.stat, b2.stat, train_x.stat = map(describe, (w1.g, w2.g, b1.g, b2.g, train_x.g))

In [80]:
w1.stat, w2.stat, b1.stat, b2.stat, train_x.stat

((tensor(6.5712), tensor(24845.4062), 'torch.FloatTensor'),
 (tensor(-2745.2048), tensor(14009.0879), 'torch.FloatTensor'),
 (tensor(-432.0642), tensor(40397.9258), 'torch.FloatTensor'),
 (tensor(-23525.9473), tensor(nan), 'torch.FloatTensor'),
 (tensor(-0.0186), tensor(0.6076), 'torch.FloatTensor'))